<a href="https://colab.research.google.com/github/giovxna/educa.lab/blob/main/educa_lab_Imers%C3%A3o_IA_Alura.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q google-genai google-adk

In [ ]:
import os
import textwrap
import warnings
from datetime import date
from google.colab import userdata
from IPython.display import display, Markdown
from google import genai
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types

In [ ]:
# Configuração da API
os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
client = genai.Client()

In [ ]:
warnings.filterwarnings("ignore")
client = genai.Client()
MODEL_ID = "gemini-2.0-flash"

In [ ]:
# Função auxiliar para executar agente

def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

def call_agent(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
            for part in event.content.parts:
                if part.text is not None:
                    final_response += part.text + "\n"
    return final_response

In [ ]:
#Criando Agentes

def criar_agente_explicador(modo_usuario):
    instrucoes = {
        "infantil": "Explique como se o usuário tivesse 5 anos. Use analogias com brinquedos, animais e situações do cotidiano de uma criança.",
        "ensino_medio": "Explique como para um estudante do ensino médio, com exemplos aplicados, mas sem termos muito técnicos.",
        "adulto": "Explique de forma clara, objetiva e técnica, como para um adulto curioso.",
        "fofoca": "Explique como se fosse uma fofoca entre amigos. Use humor, gírias leves e analogias engraçadas como se estivesse contando um babado.",
        "memes": "Explique com tom de meme e linguagem da internet. Use emojis (mas de forma moderada), hashtags e expressões engraçadas."
    }

    return Agent(
        name="agente_explicador",
        model=MODEL_ID,
        instruction=instrucoes.get(modo_usuario, "Explique de forma clara e divertida."),
        description="Agente que explica conceitos com o estilo desejado."
    )

def agente_quiz(assunto):
    quiz = Agent(
        name="agente_quiz",
        model=MODEL_ID,
        instruction="Crie uma pergunta ou desafio simples para testar se o usuário aprendeu o conteúdo.",
        description="Agente que cria um quiz ou desafio."
    )
    return call_agent(quiz, f"Crie uma pergunta para testar o conhecimento sobre: {assunto}")

def agente_link_alura(assunto):
    link = Agent(
        name="agente_alura",
        model=MODEL_ID,
        instruction="Sugira um curso da Alura que aprofunde esse tema.",
        description="Agente que sugere cursos da Alura."
    )
    return call_agent(link, f"Sugira um curso da Alura para aprender mais sobre: {assunto}")

In [ ]:
def aprofundar_assunto(explicador, assunto):
    while True:
        print("\n🔍 O que deseja fazer agora?")
        print("1 - Fazer uma nova pergunta sobre esse tema ❓")
        print("2 - Receber uma explicação mais profunda 🧠")
        print("3 - Tentar um desafio mais difícil sobre isso 🎯")
        print("4 - Ver outro assunto 🔁")
        print("5 - Encerrar 🚪")

        escolha = input("Escolha uma opção (1-5): ").strip()

        if escolha == "1":
            pergunta = input("Digite sua nova dúvida: ")
            prompt = f"Continuando sobre '{assunto}': {pergunta}"
            resposta = call_agent(explicador, prompt)
            print("\n📌 Resposta personalizada:\n")
            display(to_markdown(resposta))

        elif escolha == "2":
            prompt = f"Explique o conceito de '{assunto}' com mais profundidade e exemplos avançados."
            explicacao = call_agent(explicador, prompt)
            print("\n🔎 Explicação mais avançada:\n")
            display(to_markdown(explicacao))

        elif escolha == "3":
            quiz_dificil = Agent(
                name="quiz_dificil",
                model=MODEL_ID,
                instruction="Crie um desafio mais difícil, como um quiz com raciocínio ou aplicação prática.",
                description="Quiz avançado"
            )
            desafio = call_agent(quiz_dificil, f"Crie um desafio difícil sobre: {assunto}")
            print("\n🧩 Desafio avançado:\n")
            display(to_markdown(desafio))

        elif escolha == "4":
            break  # Voltar para o menu principal

        elif escolha == "5":
            print("\n👋 Até logo, bons estudos!")
            exit()

        else:
            print("Opção inválida. Tente novamente.")


In [ ]:
def educa_lab(assunto, tempo, modo):
    explicador = criar_agente_explicador(modo)
    prompt = f"Explique o conceito de '{assunto}' em no máximo {tempo} minutos."
    explicacao = call_agent(explicador, prompt)
    quiz = agente_quiz(assunto)
    link = agente_link_alura(assunto)

    print("\n📚 Explicação personalizada:\n")
    display(to_markdown(explicacao))

    print("\n❓ Pergunta rápida para testar aprendizado:\n")
    display(to_markdown(quiz))

    print("\n📎 Para saber mais:\n")
    display(to_markdown(link))

    # Chamada para aprofundamento
    aprofundar_assunto(explicador, assunto)

# ---------- Execução principal ---------- #
def iniciar_educa_lab():
    while True:
        assunto = input("\nQuero aprender sobre: ")
        tempo = input("Tempo que tenho (2, 5, 10, 15 minutos): ")

        print("\n🧠 Como você quer aprender?")
        print("1 - Como se eu tivesse 5 anos 👶")
        print("2 - Como estudante do ensino médio 🧑")
        print("3 - Como adulto curioso 👩‍🎓")
        print("4 - Com analogias e fofoca 🤪")
        print("5 - Em memes 😂")
        print("6 - Surpreenda-me com o estilo! 🎲")

        modo_map = {
            "1": "infantil",
            "2": "ensino_medio",
            "3": "adulto",
            "4": "fofoca",
            "5": "memes"
        }

        modo_escolhido = input("Escolha um número: ")

        if modo_escolhido == "6":
            import random
            modo = random.choice(list(modo_map.values()))
            print(f"✨ Estilo sorteado: {modo}")
        else:
            modo = modo_map.get(modo_escolhido, "adulto")

        educa_lab(assunto, tempo, modo)

        repetir = input("\n🔁 Quer aprender outro assunto? (s/n): ").lower()
        if repetir != 's':
            print("\nAté a próxima! 👋")
            break

In [ ]:
# Executar o app
iniciar_educa_lab()